In [ ]:
from pyspark.sql import SparkSession, Row

# Initialize Spark session
spark = SparkSession.builder.appName("product_orders_analytics").getOrCreate()

# Sample dataset for orders
orders_data = [
    Row(order_id=1, customer_name="Alice", product="Smartphone", category="Electronics", quantity=2, unit_price=500, order_date="2023-01-05"),
    Row(order_id=2, customer_name="Bob", product="T-Shirt", category="Clothing", quantity=4, unit_price=20, order_date="2023-01-15"),
    Row(order_id=3, customer_name="Charlie", product="Bookshelf", category="Furniture", quantity=1, unit_price=12000, order_date="2023-02-01"),
    Row(order_id=4, customer_name="David", product="Laptop", category="Electronics", quantity=1, unit_price=1500, order_date="2023-01-20"),
    Row(order_id=5, customer_name="Eva", product="Notebook", category="Books", quantity=5, unit_price=10, order_date="2023-03-12"),
    Row(order_id=6, customer_name="Frank", product="Sofa", category="Furniture", quantity=1, unit_price=18000, order_date="2023-04-10"),
    Row(order_id=7, customer_name="Grace", product="Jeans", category="Clothing", quantity=2, unit_price=40, order_date="2023-01-25"),
    Row(order_id=8, customer_name="Hannah", product="Headphones", category="Electronics", quantity=3, unit_price=150, order_date="2023-05-03"),
    Row(order_id=9, customer_name="Ian", product="Desk", category="Furniture", quantity=1, unit_price=9500, order_date="2023-03-17"),
    Row(order_id=10, customer_name="Jane", product="Novel", category="Books", quantity=3, unit_price=15, order_date="2023-02-23"),
    Row(order_id=11, customer_name="Kyle", product="Monitor", category="Electronics", quantity=2, unit_price=300, order_date="2023-01-11"),
    Row(order_id=12, customer_name="Liam", product="Jacket", category="Clothing", quantity=1, unit_price=70, order_date="2023-04-28"),
]

# Create DataFrame
orders_df = spark.createDataFrame(orders_data)
orders_df.createOrReplaceTempView("orders_local")
orders_df.createOrReplaceGlobalTempView("orders_global")
orders_df.show(truncate=False)

In [ ]:
# 1. Electronics orders with quantity >= 2
SELECT * FROM orders_local WHERE category = 'Electronics' AND quantity >= 2


In [ ]:
# 2. TotalAmount per order
SELECT *, quantity * unit_price AS total_amount FROM orders_local


In [ ]:
# 3. Total number of orders per category
SELECT category, COUNT(*) AS total_orders FROM orders_local GROUP BY category


In [ ]:
# 4. Orders placed in January 2023
SELECT * FROM orders_local WHERE order_date LIKE '2023-01%'


In [ ]:
# 5. Average unit price per category
SELECT category, AVG(unit_price) AS avg_unit_price FROM orders_local GROUP BY category


In [ ]:
# 6. Order with the highest total amount
SELECT *, quantity * unit_price AS total_amount FROM orders_local ORDER BY total_amount DESC LIMIT 1


In [ ]:
# 7. Drop local view and attempt query
spark.catalog.dropTempView('orders_local')
# spark.sql('SELECT * FROM orders_local').show()  


In [ ]:
# 1. Furniture orders with total amount > 10000
SELECT *, quantity * unit_price AS total_amount FROM global_temp.orders_global WHERE category = 'Furniture' AND (quantity * unit_price) > 10000


In [ ]:
# 2. Add DiscountFlag column
SELECT *, CASE WHEN quantity > 3 THEN 'Yes' ELSE 'No' END AS discount_flag FROM global_temp.orders_global


In [ ]:
# 3. Customers who ordered more than 1 product type
SELECT customer_name FROM global_temp.orders_global GROUP BY customer_name HAVING COUNT(DISTINCT product) > 1


In [ ]:
# 4. Orders count per month
SELECT SUBSTRING(order_date, 1, 7) AS order_month, COUNT(*) AS order_count FROM global_temp.orders_global GROUP BY order_month


In [ ]:
# 5. Rank products by total quantity sold
SELECT product, SUM(quantity) AS total_quantity, RANK() OVER (ORDER BY SUM(quantity) DESC) AS rank
FROM global_temp.orders_global
GROUP BY product


In [ ]:
# 6. New session global view query
new_spark = SparkSession.builder.appName("new_session").getOrCreate()
new_spark.sql("SELECT * FROM global_temp.orders_global").show()


In [ ]:
# 1. Save 'Books' category as global view
books_df = spark.sql("SELECT * FROM global_temp.orders_global WHERE category = 'Books'")
books_df.createOrReplaceGlobalTempView("books_orders")


In [ ]:
# 2. Most purchased product per category
SELECT category, product, SUM(quantity) AS total_quantity FROM global_temp.orders_global GROUP BY category, product ORDER BY category, total_quantity DESC


In [ ]:
# 3. Create 'filtered_orders' view without 'Clothing'
filtered_df = spark.sql("SELECT * FROM global_temp.orders_global WHERE category != 'Clothing'")
filtered_df.createOrReplaceTempView("filtered_orders")
